# Bloque C: Análisis del 'Gran Juego' y Desarrollo Post-Soviético

## Pregunta de Investigación
> **"¿Son los factores de 'Poder Duro' (Gasto Militar) o de 'Poder Blando' (Democracia, Control de Corrupción) los que determinan el desarrollo económico en la periferia post-soviética?"**

Este análisis utilizará **Apache Spark** para procesar los datos y un modelo de Machine Learning **Random Forest** para determinar la importancia de cada variable.

In [ ]:
# Inicialización de Spark y Librerías
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc, corr
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Crear sesión Spark explícita
spark = SparkSession.builder \
    .appName("Analisis Gran Juego ML") \
    .getOrCreate()

print("Sesión de Spark iniciada correctamente.")

# Configurar estilo de gráficos
sns.set_theme(style="whitegrid")

## 1. Carga de Datos Procesados

In [ ]:
# Cargar datos Parquet (generados en el paso anterior)
DATA_PATH = "/home/jovyan/work/data/processed/qog_great_game.parquet"

df = spark.read.parquet(DATA_PATH)
print(f"Datos cargados. Total registros: {df.count()}")
df.printSchema()
df.show(5)

## 2. Análisis Descriptivo (Spark SQL)

In [ ]:
# Crear vista temporal para SQL
df.createOrReplaceTempView("qog")

# Consulta 1: Promedios por País (Ordenados por PIB)
query_avg = """
SELECT 
    cname, 
    ROUND(AVG(gle_cgdpc), 2) as avg_gdp, 
    ROUND(AVG(wdi_lifexp), 1) as avg_lifexp,
    ROUND(AVG(p_polity2), 1) as avg_democracy,
    ROUND(AVG(wdi_expmil), 2) as avg_military_exp
FROM qog
GROUP BY cname
ORDER BY avg_gdp DESC
"""
spark.sql(query_avg).show()

In [ ]:
# Matriz de Correlación (Pandas para visualización)
features = ['gle_cgdpc', 'wdi_lifexp', 'p_polity2', 'vdem_corr', 'wdi_expmil']
df_corr = df.select(features).toPandas().dropna()

plt.figure(figsize=(10, 8))
sns.heatmap(df_corr.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Matriz de Correlación: Factores Políticos vs Económicos")
plt.show()

## 3. Machine Learning: Random Forest Regressor

Entrenaremos un modelo para predecir el PIB (`gle_cgdpc`) basándonos en las variables de estabilidad y poder.

In [ ]:
# 1. Preprocesamiento: Eliminar nulos para ML
# Random Forest no maneja NaNs nativamente en Spark MLlib antiguo, mejor limpiar
ml_cols = ['gle_cgdpc', 'wdi_lifexp', 'p_polity2', 'vdem_corr', 'wdi_expmil']
df_ml = df.select(ml_cols).dropna()

print(f"Registros aptos para ML tras limpieza: {df_ml.count()}")

# 2. Vector Assembler
feature_cols = ['wdi_lifexp', 'p_polity2', 'vdem_corr', 'wdi_expmil']
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# 3. Definir Modelo
rf = RandomForestRegressor(featuresCol="features", labelCol="gle_cgdpc", numTrees=100, seed=42)

# 4. Pipeline
pipeline = Pipeline(stages=[assembler, rf])

# 5. Entrenar
model = pipeline.fit(df_ml)
print("Modelo Random Forest entrenado exitosamente.")

## 4. Interpretación: Feature Importance (Importancia de Variables)

In [ ]:
# Extraer importancia
rf_model = model.stages[-1]
importances = rf_model.featureImportances

# Crear DataFrame para visualizar
importances_list = []
for i, col in enumerate(feature_cols):
    importances_list.append({"Feature": col, "Importance": importances[i]})

df_imp = pd.DataFrame(importances_list).sort_values(by="Importance", ascending=False)

# Visualizar
plt.figure(figsize=(10, 6))
sns.barplot(x="Importance", y="Feature", data=df_imp, palette="viridis")
plt.title("Random Forest: ¿Qué define la economía en el Gran Juego?")
plt.xlabel("Importancia Relativa (0-1)")
plt.ylabel("Variables")
plt.show()

### Conclusión Preliminar
Observando el gráfico de arriba:
- Si **`wdi_expmil`** es alta: Hipótesis de "Poder Duro" (Economía de Guerra/Seguridad).
- Si **`p_polity2`** o **`vdem_corr`** son altas: Hipótesis de "Poder Blando" (Instituciones estables).
- Nota: `wdi_lifexp` suele tener alta correlación endógena con el PIB, por lo que podría dominar. Es importante analizar la segunda variable más fuerte.